<a id='top'></a>
## Power calculations concept

1. Read in the original pilot study data to determine % qualifiers passed.
2. Read in a cleaned, reshaped set of data. This data has removed the failed qualifiers.
3. Establish a power calculator function. Test with data from FE pg 93.
4. Conduct analysis of control vs nonsense names
5. Conduct analysis of control vs terrible names
6. Average Treatment Effect is set at .5
6. $\alpha$ will be set at .05
7. Power will be $\ge$ .8

#### Methods used to increase power in simulations
1. Manipulated sample size.

### Results summary:

__Bottom line: Minimum of 130 surveys needed__

This particular data set had no incorrect qualifying answers. Since none were wrong, I used the % of wrong qualifiers from the previous data - 77%

While we hit 80% power at 400 actual surveys taken, need to take into account the number of subjects who do not pass the qualifiers. Assuming that is still approximately 77%, we get the following results:

1. Control v. Nonsense: $\frac{400}{.77} = 519$ (round to 520)

2. Control v. Terrible: $\frac{400}{.77} = 519$ (round to 520)

This would be 780 observations (260 * 3). Given there are 6 questions per survey, we can calculate that to be $\frac{780}{6} = 130$ subjects needed. 

__For us to detect an ATE of .5 at a 95% confidence level 80% of the time we ran this experiment, we would need at least 130 surveys__.



[Control and Nonsense](#nonsense)

[Control and Terrible](#terrible)


In [22]:
library(data.table)

# read in original data to calculate % qualifying questions correct.
pilot.original <- data.table(read.csv("final_test_data.csv"))

# read in the cleaned, reshaped data set
pilot.data <- data.table(read.csv("final_test_data_rs.csv"))

# power calculator function. Given ATE, sigma, alpha, and N,
# calculates the power.
power_calculator <- function(ate=.5, sigma, alpha=0.05, N) { 

    lowertail <- (abs(ate)*sqrt(N))/(2*sigma) 
    uppertail <- -1*lowertail 
    beta <- pnorm(lowertail- qnorm(1-alpha/2), lower.tail=TRUE) + 
        1- pnorm(uppertail- qnorm(1-alpha/2), lower.tail=FALSE) 
    return(beta) 
}

# function check - inputs are from FE, pg 93. Result = .80 as per book.
print(paste('Power should be .8. Result = ', 
            round(power_calculator(5, 20, .05, 500), digits=2)))

# id the wrong answers to the qualfying questions
pilot.original <-pilot.original[,  QID2.wrong:= .(QID2 != "3")]
pilot.original <-pilot.original[,  QID3.wrong:= .(QID3 != "Wednesday")]
pilot.original <-pilot.original[,  QID4.wrong:= .(QID4 != "Obama")]

# calc the % of wrong answers
bad.quals <- nrow(pilot.original[(QID2.wrong | QID3.wrong |  QID4.wrong)]) / nrow(pilot.original)
print(paste('Percent of wrong qualifiers: ', round(bad.quals, digits=2) * 100))

# calc the % of correct answers
good.quals <- 1 - bad.quals
print(paste('Percent of correct qualifiers: ', round(good.quals, digits=2) * 100))

good.quals <- .77 # keeping previous assumption given there were no incorrect responses in this data.
print(paste('Adjusted percent of correct qualifiers: ', round(good.quals, digits=2) * 100))

[1] "Power should be .8. Result =  0.8"
[1] "Percent of wrong qualifiers:  0"
[1] "Percent of correct qualifiers:  100"
[1] "Adjusted percent of correct qualifiers:  77"


<a id='nonsense'></a>
## Control and Nonsense Names

[Return to Top](#top)

In [21]:
# read in the data into its separate groups of control name, nonsense name
control.data <- pilot.data[treatment == 0]
treat1.data <- pilot.data[treatment == 1]

# sample sizes to test
sample_sizes <- seq(from=375, to=425, by=25)

# function to calculate power for a randomized sampling of data
betas <- function(sampl){ 
        
    # sample from the control subset
    x <- control.data[sample(1:nrow(control.data), size=sampl, 
                            replace=TRUE)]
    

    # use pooled sd for the rating by funding category
    sigma_x <- sqrt((var(x[funding_id == 0]$rating) +
             var(x[funding_id == 1]$rating) +
             var(x[funding_id == 2]$rating) +
             var(x[funding_id == 4]$rating) +
             var(x[funding_id == 5]$rating) +
             var(x[funding_id == 6]$rating) +
             var(x[funding_id == 7]$rating)) / 7) 

    # get number of rows in control sample
    n_x <- nrow(x)
    
    # sample the treatment 1 (nonsense name) dataset
    y <- treat1.data[sample(1:nrow(treat1.data), size=sampl, 
                            replace=TRUE)]
    
    # use pooled sd for the rating by funding category
    sigma_y <- sqrt((var(y[funding_id == 0]$rating) +
             var(y[funding_id == 1]$rating) +
             var(y[funding_id == 2]$rating) +
             var(y[funding_id == 3]$rating) +
             var(y[funding_id == 4]$rating) +
             var(y[funding_id == 6]$rating) +
             var(y[funding_id == 7]$rating)) / 7)
    
    # calculate the avg std deviation for the formula
    sigma <- mean(sigma_x, sigma_y)
    
    # get number of rows in nonsense name group
    n_y <- nrow(y)
    
    # number of total rows
    n <- n_x + n_y
    
    # calculate power
    ctl_t1 <- power_calculator(sigma = sigma, N=n)
    
    # return the beta (power) value
    ctl_t1
}

for (sampl in sample_sizes){
    
    print(paste('Sample size: ', sampl))

    beta_sims <- replicate(1000, betas(sampl))
        
    print(paste('Avg Power Calc: ', mean(beta_sims)))
        
    sample_size_needed = sampl / good.quals
    print(paste('Sample size needed factoring in % bad qualifying answers: ',
                round(sample_size_needed, digits = 0)))

    cat('\n')
}


[1] "Sample size:  375"
[1] "Avg Power Calc:  0.788231666159307"
[1] "Sample size needed factoring in % bad qualifying answers:  487"

[1] "Sample size:  400"
[1] "Avg Power Calc:  0.81375766780837"
[1] "Sample size needed factoring in % bad qualifying answers:  519"

[1] "Sample size:  425"
[1] "Avg Power Calc:  0.836083483737591"
[1] "Sample size needed factoring in % bad qualifying answers:  552"



<a id='terrible'></a>
## Control and Terrible Names

[Return to Top](#top)

In [20]:
treat2.data <- pilot.data[treatment == 2]
sample_sizes <- seq(from=375, to=425, by=25)

betas <- function(sampl){
    
    # sample from the subset given the provided sample size
    x <- control.data[sample(1:nrow(control.data), size=sampl, 
                            replace=TRUE)]

    # use pooled sd for sigma
    sigma_x <- sqrt((var(x[funding_id == 0]$rating) +
             var(x[funding_id == 1]$rating) +
             var(x[funding_id == 2]$rating) +
             var(x[funding_id == 4]$rating) +
             var(x[funding_id == 5]$rating) +
             var(x[funding_id == 6]$rating) +
             var(x[funding_id == 7]$rating)) / 7) 

    n_x <- nrow(x)
    
    # sample the treatment 2 (terrible name) dataset
    y <- treat2.data[sample(1:nrow(treat2.data), size=sampl, 
                            replace=TRUE)]
    
    # use pooled sd for the rating by funding category
    sigma_y <- sqrt((var(y[funding_id == 0]$rating) +
             var(y[funding_id == 1]$rating) +
             var(y[funding_id == 2]$rating) +
             var(y[funding_id == 3]$rating) +
             var(y[funding_id == 4]$rating) +
             var(y[funding_id == 5]$rating) +
             var(y[funding_id == 6]$rating) +
             var(y[funding_id == 7]$rating)) / 8)
    
    # calculate the avg std deviation for the formula
    sigma <- mean(sigma_x, sigma_y)
    
    # get number of rows in terrible name group
    n_y <- nrow(y)
    
    # number of total rows
    n <- n_x + n_y
    
    # calculate power
    ctl_t2 <- power_calculator(sigma = sigma, N=n)
    
    # return the beta (power) value
    ctl_t2
}

for (sampl in sample_sizes){
    
    print(paste('Sample size: ', sampl))

    beta_sims <- replicate(1000, betas(sampl))
    
    print(paste('Avg Power Calc: ', mean(beta_sims)))
        
    sample_size_needed = sampl / good.quals
    print(paste('Sample size needed factoring in % bad qualifying answers: ', round(sample_size_needed, digits = 0)))

    cat('\n')
}


[1] "Sample size:  375"
[1] "Avg Power Calc:  0.787908474861802"
[1] "Sample size needed factoring in % bad qualifying answers:  487"

[1] "Sample size:  400"
[1] "Avg Power Calc:  0.813268023176872"
[1] "Sample size needed factoring in % bad qualifying answers:  519"

[1] "Sample size:  425"
[1] "Avg Power Calc:  0.834816842278386"
[1] "Sample size needed factoring in % bad qualifying answers:  552"

